In [1]:
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [2]:
data_movies=pd.read_csv('tmdb_5000_movies.csv')
data_credit=pd.read_csv('tmdb_5000_credits.csv')
data_credit=data_credit.rename(index=str,columns={'movie_id':'id'})
data=data_movies.merge(data_credit,on='id')

In [3]:
data=data.drop(columns=['homepage','title_x','title_y','status','production_countries'],axis=1)

In [4]:
tfv = TfidfVectorizer(min_df=3,  max_features=None,
                      strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
                      ngram_range=(1, 3),
                      stop_words='english')
data['overview'].fillna('', inplace=True)

tfv_m = tfv.fit_transform(data['overview'])
sigm = sigmoid_kernel(tfv_m, tfv_m)
indices = pd.Series(data.index,
                    index=data['original_title']).drop_duplicates()

In [5]:
def give_recomendations(title, sig=sigm):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    sig_scores = sig_scores[1:11]
    movie_indices = [i[0] for i in sig_scores]
    return data['original_title'].iloc[movie_indices]


In [6]:
# so here I have a problem to work with text, if I don't put accurate key I won't get anything
print(give_recomendations('Avatar'))

1341                Obitaemyy Ostrov
634                       The Matrix
3604                       Apollo 18
2130                    The American
775                        Supernova
529                 Tears of the Sun
151                          Beowulf
311     The Adventures of Pluto Nash
847                         Semi-Pro
942                 The Book of Life
Name: original_title, dtype: object
